

# `ibis` mutates and aggregates

### Learning Goals

- use `group_by()` and `aggregate()` patterns to summarize data
- use `mutate()` to create an new column that is a function of data from existing columns in a table.
- use `order_by()` to arrange rows by one or more columns.

In [4]:
import ibis
from ibis import _
import ibis.selectors as s

con = ibis.duckdb.connect()

In [5]:
base_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/"

stock = con.read_csv(base_url + "stock.csv", nullstr="NA")
timeseries = con.read_csv(base_url + "timeseries.csv", nullstr="NA")
assessment = con.read_csv(base_url + "assessment.csv", nullstr="NA")


# Stock "assessments"

The next thing we need to know is a stock assessment.  

In [6]:
assessment

DatabaseTable: ibis_read_csv_ldxnvs2lbzcblprbeqrxubgcye
  assessid       string
  assessorid     string
  stockid        string
  stocklong      string
  recorder       string
  daterecorded   timestamp(6)
  dateloaded     timestamp(6)
  assessyear     string
  assesssource   string
  contacts       string
  notes          string
  pdffile        string
  assess         int64
  refpoints      int64
  assessmethod   string
  assesscomments string
  xlsfilename    string
  mostrecent     int64

Are some `stockid`s assessed multiple times? One intuitive idea is to `filter()` for a single stockid, and see if we get back multiple rows (multiple assessments).  Let's take a look at "COD2J3KL":

In [8]:
(assessment
 .filter(_.stockid == "COD2J3KL")
 .execute()
)

,assessid,assessorid,stockid,stocklong,recorder,daterecorded,dateloaded,assessyear,assesssource,contacts,notes,pdffile,assess,refpoints,assessmethod,assesscomments,xlsfilename,mostrecent
0,DFO-COD2J3KL-1959-2014-WATSON,DFO,COD2J3KL,Atlantic cod Southern Labrador-Eastern Newfoun...,WATSON,2016-02-25,2016-02-18,1959-2014,http://www.dfo-mpo.gc.ca/csas-sccs/publication...,NULL,NULL,NULL,1,1,SURBA,NULL,WATSON_COD2J3KL_2014-DH-edited.xlsx,0
1,DFO-NFLD-COD2J3KL-1850-2011-CHING,DFO-NFLD,COD2J3KL,Atlantic cod Southern Labrador-Eastern Newfoun...,CHING,2013-10-22,2001-12-31,1850-2011,NULL,NULL,NULL,Stock Assesment of Northern (2J3KL) Cod in 2013,1,1,SURBA,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-COD2J3...,0
2,DFO-NFLD-COD2J3KL-1959-2018-ASHBROOK,DFO-NFLD,COD2J3KL,Atlantic cod Southern Labrador-Eastern Newfoun...,ASHBROOK,2019-06-20,2019-08-22,1959-2018,http://www.dfo-mpo.gc.ca/csas-sccs/Publication...,NULL,NULL,AtlanticCod.2J3KL.pdf,1,1,ICA,NULL,ASHBROOK-COD2J3KL-2018-DH-edited.xlsx,0
3,DFO-NFLD-COD2J3KL-1959-2021-HIVELY,DFO-NFLD,COD2J3KL,Atlantic cod Southern Labrador-Eastern Newfoun...,HIVELY,2023-11-08,2023-11-10,1959-2021,NULL,NULL,Science Advisory Report 2022/041,NULL,1,1,ICA,NULL,CND_vers4.62_updates.xlsx,999


Indeed, it looks like their are four assessments of this stock, each conducted in different years and spanning different periods in time!  `filter()`ing for each possible stockid would be tedious though. These four assessments that correspond to this stockid 

In [20]:
(assessment
 .group_by(_.stockid)
 .agg(n=_.count())
 .execute()
)

,stockid,n
0,CODIIIaW-IV-VIId,12
1,HERRNIRS,11
2,HERRNS-IIIa-VIId,11
3,POLLFAPL,11
4,WHITNS-VIId,11
...,...,...
1507,SSALMQUIN,1
1508,SPRAYVI-VIIbj,1
1509,TBSKAIXa,1
1510,TBSKAVI,1


In [24]:
assessment.group_by(_.stockid).order_by(_.assessyear).agg(assessid=_.assessid.last()).filter(_.stockid == "CODIIIaW-IV-VIId").execute()

,stockid,assessid
0,CODIIIaW-IV-VIId,WGNSSK-CODIIIaW-IV-VIId-1963-2021-ICESIMP2021-2


In [ ]:
timeseries

In [13]:
stock

DatabaseTable: ibis_read_csv_aee33mgu3rfsnammlaymycd2lu
  stockid         string
  tsn             int64
  scientificname  string
  commonname      string
  areaid          string
  stocklong       string
  region          string
  primary_country string
  primary_FAOarea int64
  ISO3_code       string
  GRSF_uuid       string
  GRSF_areaid     string
  inmyersdb       int64
  myersstockid    string
  state           string